In [1]:
import os
os.environ["SPARK_HOME"] = "/Applications/spark"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_PYTHON"] = "python"

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-ml") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()

26/01/23 22:37:56 WARN Utils: Your hostname, Kavis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.24 instead (on interface en0)
26/01/23 22:37:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/kavisanthoshkumar/.ivy2/cache
The jars for the packages stored in: /Users/kavisanthoshkumar/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a8e43d7-d5c3-4a9f-8f7d-37138b455353;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Applications/Spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.4!hadoop-aws.jar (124ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.262!aws-java-sdk-bundle.jar (5008ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-openssl-1.0.7.Final.jar ...
	[SUCCESSFUL ] org.wildfly.openssl#wildfly-openssl;1.0.7.Final!wildfly-openssl.jar (151ms)
:: resolution report :: resolve 2521ms :: artifacts dl 5286ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default

In [23]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, 
    StructField, 
    IntegerType, 
    FloatType, 
    StringType, 
    DoubleType, 
    BooleanType
)
from pyspark.sql import DataFrame

#### 1.	Load the CSV file with header and infer schema.

In [10]:
data = spark.read.csv("../dataset/online_shoppers_intention.csv", header = True, inferSchema = True)

data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



#### 2.	Load the CSV again using an explicitly defined schema.

In [17]:
data_schema = StructType([
    StructField("Administrative", IntegerType()), 
    StructField("Administrative_Duration", DoubleType()), 
    StructField("Informational", IntegerType()), 
    StructField("Informational_Duration", DoubleType()), 
    StructField("ProductRelated", IntegerType()), 
    StructField("ProductRelated_Duration", DoubleType()), 
    StructField("BounceRates", DoubleType()), 
    StructField("ExitRates", DoubleType()), 
    StructField("PageValues", DoubleType()), 
    StructField("SpecialDay", DoubleType()),
    StructField("Month", StringType()), 
    StructField("OperatingSystems", IntegerType()), 
    StructField("Browser", IntegerType()),
    StructField("Region", IntegerType()), 
    StructField("TrafficType",IntegerType()), 
    StructField("VisitorType", StringType()), 
    StructField("Weekend", BooleanType()),
    StructField("Revenue", BooleanType())
])


data = spark.read.csv("../dataset/online_shoppers_intention.csv", schema = data_schema, header = True)

data.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



#### 3.	Add a column ingestion_timestamp using current timestamp.

In [34]:
current_date = datetime.now().date()
print(f"Ingestion Date : {current_date}")

Ingestion Date : 2026-01-23


In [36]:
data = data.withColumn("ingestion_timestamp", F.lit(current_date))

data.select('ingestion_timestamp').show()

+-------------------+
|ingestion_timestamp|
+-------------------+
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
|         2026-01-23|
+-------------------+
only showing top 20 rows

